### 원본 train.csv 또는 test.csv 에 결측치 없는 X, Y 컬럼 추가하기.

In [49]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

In [50]:
isTrainSelected = 0

if isTrainSelected:
    origin_filepath = '../train.csv'
    target_filepath = 'train_add_newXY.csv'
else:
    origin_filepath = '../test.csv'
    target_filepath = 'test_add_newXY.csv'

In [51]:
dt = pd.read_csv(origin_filepath)

In [52]:
# Train data와 Test data shape은 아래와 같습니다.
print('Train data shape : ', dt.shape)

Train data shape :  (1118822, 52)


In [53]:
dt[dt['번지'].isnull()]

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target
56930,서울특별시 서초구 내곡동,NaN,0.0,0.0,서초포레스타2단지,84.87,201710,16,6,2015,...,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,88900
56931,서울특별시 서초구 내곡동,NaN,0.0,0.0,서초포레스타2단지,59.21,201710,19,11,2015,...,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,73000
56932,서울특별시 서초구 내곡동,NaN,0.0,0.0,서초포레스타2단지,59.21,201710,21,6,2015,...,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,76300
56933,서울특별시 서초구 내곡동,NaN,0.0,0.0,서초포레스타2단지,84.87,201710,21,19,2015,...,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,90000
56934,서울특별시 서초구 내곡동,NaN,0.0,0.0,서초포레스타2단지,84.48,201710,24,19,2015,...,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,88000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720188,서울특별시 서초구 신원동,NaN,NaN,NaN,힐스테이트 서초 젠트리스,84.99,201504,17,9,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79000
720189,서울특별시 서초구 신원동,NaN,NaN,NaN,힐스테이트 서초 젠트리스,101.90,201505,1,6,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95000
720190,서울특별시 서초구 신원동,NaN,NaN,NaN,힐스테이트 서초 젠트리스,84.95,201506,16,6,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87200
720191,서울특별시 서초구 신원동,NaN,NaN,NaN,힐스테이트 서초 젠트리스,101.90,201506,26,8,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94500


In [54]:
dt.loc[dt['아파트명'] == '서초포레스타2단지', '번지'] = '384'
dt[dt['아파트명'] == '서초포레스타2단지']

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target
56930,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.87,201710,16,6,2015,...,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,88900
56931,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,59.21,201710,19,11,2015,...,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,73000
56932,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,59.21,201710,21,6,2015,...,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,76300
56933,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.87,201710,21,19,2015,...,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,90000
56934,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.48,201710,24,19,2015,...,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,88000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503699,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.87,201709,22,10,2015,...,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,83000
503700,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.48,201709,23,9,2015,...,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,84000
503701,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,59.21,201709,25,20,2015,...,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,73000
605309,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.48,201605,31,10,2015,...,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,83000


In [55]:
dt[dt['번지'].isnull()]

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target
60194,서울특별시 서초구 신원동,NaN,NaN,NaN,힐스테이트 서초 젠트리스,114.90,201801,6,6,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,147000
60195,서울특별시 서초구 신원동,NaN,NaN,NaN,힐스테이트 서초 젠트리스,114.90,201801,8,4,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134000
60196,서울특별시 서초구 신원동,NaN,NaN,NaN,힐스테이트 서초 젠트리스,84.99,201801,8,1,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114000
60197,서울특별시 서초구 신원동,NaN,NaN,NaN,힐스테이트 서초 젠트리스,84.95,201801,12,3,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109250
60198,서울특별시 서초구 신원동,NaN,NaN,NaN,힐스테이트 서초 젠트리스,114.90,201801,24,7,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720188,서울특별시 서초구 신원동,NaN,NaN,NaN,힐스테이트 서초 젠트리스,84.99,201504,17,9,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79000
720189,서울특별시 서초구 신원동,NaN,NaN,NaN,힐스테이트 서초 젠트리스,101.90,201505,1,6,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95000
720190,서울특별시 서초구 신원동,NaN,NaN,NaN,힐스테이트 서초 젠트리스,84.95,201506,16,6,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87200
720191,서울특별시 서초구 신원동,NaN,NaN,NaN,힐스테이트 서초 젠트리스,101.90,201506,26,8,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94500


In [56]:
dt.loc[dt['아파트명'] == '힐스테이트 서초 젠트리스', '번지'] = '557'
dt[dt['아파트명'] == '힐스테이트 서초 젠트리스']

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target
60194,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,114.90,201801,6,6,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,147000
60195,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,114.90,201801,8,4,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134000
60196,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,84.99,201801,8,1,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114000
60197,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,84.95,201801,12,3,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109250
60198,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,114.90,201801,24,7,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720188,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,84.99,201504,17,9,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79000
720189,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,101.90,201505,1,6,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95000
720190,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,84.95,201506,16,6,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87200
720191,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,101.90,201506,26,8,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94500


In [57]:
dt[dt['번지'].isnull()]

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target


In [58]:
dt['지번주소'] = (dt['시군구'] + " " + dt['번지']).astype(str)
dt

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,지번주소
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,124000,서울특별시 강남구 개포동 658-1
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,...,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,123500,서울특별시 강남구 개포동 658-1
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,201712,28,5,1987,...,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,91500,서울특별시 강남구 개포동 658-1
3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,3,4,1987,...,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,130000,서울특별시 강남구 개포동 658-1
4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,8,2,1987,...,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,117000,서울특별시 강남구 개포동 658-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118817,서울특별시 은평구 구산동,382,382.0,0.0,갈현현대,59.94,200707,12,11,1998,...,366.0,의무,2013-06-04 16:18:51.0,Y,N,126.905638,37.612962,2013-03-07 09:46:27.0,20000,서울특별시 은평구 구산동 382
1118818,서울특별시 은평구 구산동,382,382.0,0.0,갈현현대,59.94,200708,25,10,1998,...,366.0,의무,2013-06-04 16:18:51.0,Y,N,126.905638,37.612962,2013-03-07 09:46:27.0,20000,서울특별시 은평구 구산동 382
1118819,서울특별시 은평구 구산동,382,382.0,0.0,갈현현대,84.83,200708,31,20,1998,...,366.0,의무,2013-06-04 16:18:51.0,Y,N,126.905638,37.612962,2013-03-07 09:46:27.0,28000,서울특별시 은평구 구산동 382
1118820,서울특별시 은평구 구산동,382,382.0,0.0,갈현현대,84.83,200709,15,8,1998,...,366.0,의무,2013-06-04 16:18:51.0,Y,N,126.905638,37.612962,2013-03-07 09:46:27.0,29000,서울특별시 은평구 구산동 382


In [59]:
dt_jibun_xy = pd.read_csv('jibun_xy_by_naver.csv')
dt_jibun_xy.drop(columns='Unnamed: 0', inplace=True)
dt_jibun_xy

,지번주소,좌표X,좌표Y
0,서울특별시 강남구 개포동 1164-12,127.052949,37.473193
1,서울특별시 강남구 개포동 1164-13,127.053094,37.473287
2,서울특별시 강남구 개포동 1164-14,127.053239,37.473382
3,서울특별시 강남구 개포동 1164-20,127.054110,37.473948
4,서울특별시 강남구 개포동 1164-25,127.054827,37.474392
...,...,...,...
8948,서울특별시 중랑구 중화동 438,127.077761,37.604575
8949,서울특별시 중랑구 중화동 450,127.081923,37.597155
8950,서울특별시 중랑구 중화동 452,127.078533,37.599890
8951,서울특별시 중랑구 중화동 453,127.080084,37.602687


In [60]:
dt = pd.merge(dt, dt_jibun_xy, on='지번주소', how='left')
dt

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,단지승인일,사용허가여부,관리비 업로드,좌표X_x,좌표Y_x,단지신청일,target,지번주소,좌표X_y,좌표Y_y
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,124000,서울특별시 강남구 개포동 658-1,127.056859,37.476276
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,...,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,123500,서울특별시 강남구 개포동 658-1,127.056859,37.476276
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,201712,28,5,1987,...,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,91500,서울특별시 강남구 개포동 658-1,127.056859,37.476276
3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,3,4,1987,...,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,130000,서울특별시 강남구 개포동 658-1,127.056859,37.476276
4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,8,2,1987,...,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,117000,서울특별시 강남구 개포동 658-1,127.056859,37.476276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118817,서울특별시 은평구 구산동,382,382.0,0.0,갈현현대,59.94,200707,12,11,1998,...,2013-06-04 16:18:51.0,Y,N,126.905638,37.612962,2013-03-07 09:46:27.0,20000,서울특별시 은평구 구산동 382,126.905543,37.612989
1118818,서울특별시 은평구 구산동,382,382.0,0.0,갈현현대,59.94,200708,25,10,1998,...,2013-06-04 16:18:51.0,Y,N,126.905638,37.612962,2013-03-07 09:46:27.0,20000,서울특별시 은평구 구산동 382,126.905543,37.612989
1118819,서울특별시 은평구 구산동,382,382.0,0.0,갈현현대,84.83,200708,31,20,1998,...,2013-06-04 16:18:51.0,Y,N,126.905638,37.612962,2013-03-07 09:46:27.0,28000,서울특별시 은평구 구산동 382,126.905543,37.612989
1118820,서울특별시 은평구 구산동,382,382.0,0.0,갈현현대,84.83,200709,15,8,1998,...,2013-06-04 16:18:51.0,Y,N,126.905638,37.612962,2013-03-07 09:46:27.0,29000,서울특별시 은평구 구산동 382,126.905543,37.612989


In [61]:
display(dt['좌표X_x'].isnull().sum())
display(dt['좌표Y_x'].isnull().sum())

display(dt['좌표X_y'].isnull().sum())
display(dt['좌표Y_y'].isnull().sum())

869670

869670

225

225

In [62]:
dt[dt['좌표X_y'].isnull()].head(20)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,단지승인일,사용허가여부,관리비 업로드,좌표X_x,좌표Y_x,단지신청일,target,지번주소,좌표X_y,좌표Y_y
56930,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.87,201710,16,6,2015,...,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,88900,서울특별시 서초구 내곡동 384,NaN,NaN
56931,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,59.21,201710,19,11,2015,...,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,73000,서울특별시 서초구 내곡동 384,NaN,NaN
56932,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,59.21,201710,21,6,2015,...,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,76300,서울특별시 서초구 내곡동 384,NaN,NaN
56933,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.87,201710,21,19,2015,...,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,90000,서울특별시 서초구 내곡동 384,NaN,NaN
56934,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.48,201710,24,19,2015,...,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,88000,서울특별시 서초구 내곡동 384,NaN,NaN
56935,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,59.09,201710,31,16,2015,...,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,75000,서울특별시 서초구 내곡동 384,NaN,NaN
56936,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.87,201711,7,4,2015,...,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,85000,서울특별시 서초구 내곡동 384,NaN,NaN
56937,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.48,201711,9,4,2015,...,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,84600,서울특별시 서초구 내곡동 384,NaN,NaN
56938,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.87,201711,9,2,2015,...,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,82000,서울특별시 서초구 내곡동 384,NaN,NaN
56939,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,59.21,201711,11,21,2015,...,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,77400,서울특별시 서초구 내곡동 384,NaN,NaN


In [63]:
dt.loc[dt['아파트명'] == '서초포레스타2단지', '좌표X_y'] = 127.0609502
dt.loc[dt['아파트명'] == '서초포레스타2단지', '좌표Y_y'] = 37.4555506
dt[dt['아파트명'] == '서초포레스타2단지']

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,단지승인일,사용허가여부,관리비 업로드,좌표X_x,좌표Y_x,단지신청일,target,지번주소,좌표X_y,좌표Y_y
56930,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.87,201710,16,6,2015,...,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,88900,서울특별시 서초구 내곡동 384,127.06095,37.455551
56931,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,59.21,201710,19,11,2015,...,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,73000,서울특별시 서초구 내곡동 384,127.06095,37.455551
56932,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,59.21,201710,21,6,2015,...,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,76300,서울특별시 서초구 내곡동 384,127.06095,37.455551
56933,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.87,201710,21,19,2015,...,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,90000,서울특별시 서초구 내곡동 384,127.06095,37.455551
56934,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.48,201710,24,19,2015,...,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,88000,서울특별시 서초구 내곡동 384,127.06095,37.455551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503699,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.87,201709,22,10,2015,...,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,83000,서울특별시 서초구 내곡동 384,127.06095,37.455551
503700,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.48,201709,23,9,2015,...,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,84000,서울특별시 서초구 내곡동 384,127.06095,37.455551
503701,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,59.21,201709,25,20,2015,...,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,73000,서울특별시 서초구 내곡동 384,127.06095,37.455551
605309,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.48,201605,31,10,2015,...,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,83000,서울특별시 서초구 내곡동 384,127.06095,37.455551


In [64]:
dt[dt['좌표X_y'].isnull()].head(20)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,단지승인일,사용허가여부,관리비 업로드,좌표X_x,좌표Y_x,단지신청일,target,지번주소,좌표X_y,좌표Y_y
60194,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,114.90,201801,6,6,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,147000,서울특별시 서초구 신원동 557,NaN,NaN
60195,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,114.90,201801,8,4,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,134000,서울특별시 서초구 신원동 557,NaN,NaN
60196,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,84.99,201801,8,1,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,114000,서울특별시 서초구 신원동 557,NaN,NaN
60197,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,84.95,201801,12,3,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,109250,서울특별시 서초구 신원동 557,NaN,NaN
60198,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,114.90,201801,24,7,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,105000,서울특별시 서초구 신원동 557,NaN,NaN
60199,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,114.90,201803,3,2,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,143000,서울특별시 서초구 신원동 557,NaN,NaN
60200,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,84.95,201803,15,3,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,120000,서울특별시 서초구 신원동 557,NaN,NaN
60201,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,101.90,201803,30,2,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,146000,서울특별시 서초구 신원동 557,NaN,NaN
60202,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,84.95,201805,30,1,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,125000,서울특별시 서초구 신원동 557,NaN,NaN
60203,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,101.90,201806,9,3,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,140000,서울특별시 서초구 신원동 557,NaN,NaN


In [65]:
dt.loc[dt['아파트명'] == '힐스테이트 서초 젠트리스', '좌표X_y'] = 127.0576544
dt.loc[dt['아파트명'] == '힐스테이트 서초 젠트리스', '좌표Y_y'] = 37.4562853
dt[dt['아파트명'] == '힐스테이트 서초 젠트리스']

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,단지승인일,사용허가여부,관리비 업로드,좌표X_x,좌표Y_x,단지신청일,target,지번주소,좌표X_y,좌표Y_y
60194,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,114.90,201801,6,6,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,147000,서울특별시 서초구 신원동 557,127.057654,37.456285
60195,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,114.90,201801,8,4,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,134000,서울특별시 서초구 신원동 557,127.057654,37.456285
60196,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,84.99,201801,8,1,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,114000,서울특별시 서초구 신원동 557,127.057654,37.456285
60197,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,84.95,201801,12,3,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,109250,서울특별시 서초구 신원동 557,127.057654,37.456285
60198,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,114.90,201801,24,7,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,105000,서울특별시 서초구 신원동 557,127.057654,37.456285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720188,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,84.99,201504,17,9,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,79000,서울특별시 서초구 신원동 557,127.057654,37.456285
720189,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,101.90,201505,1,6,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,95000,서울특별시 서초구 신원동 557,127.057654,37.456285
720190,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,84.95,201506,16,6,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,87200,서울특별시 서초구 신원동 557,127.057654,37.456285
720191,서울특별시 서초구 신원동,557,NaN,NaN,힐스테이트 서초 젠트리스,101.90,201506,26,8,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,94500,서울특별시 서초구 신원동 557,127.057654,37.456285


In [66]:
display(dt['번지'].isnull().sum())
display(dt['지번주소'].isnull().sum())
display(dt['좌표X_y'].isnull().sum())
display(dt['좌표Y_y'].isnull().sum())

0

0

0

0

In [67]:
dt.rename(columns={'좌표X_x':'좌표X', '좌표Y_x':'좌표Y', '좌표X_y':'좌표X_2', '좌표Y_y':'좌표Y_2'}, inplace=True)
dt

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,지번주소,좌표X_2,좌표Y_2
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,124000,서울특별시 강남구 개포동 658-1,127.056859,37.476276
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,...,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,123500,서울특별시 강남구 개포동 658-1,127.056859,37.476276
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,201712,28,5,1987,...,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,91500,서울특별시 강남구 개포동 658-1,127.056859,37.476276
3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,3,4,1987,...,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,130000,서울특별시 강남구 개포동 658-1,127.056859,37.476276
4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,8,2,1987,...,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,117000,서울특별시 강남구 개포동 658-1,127.056859,37.476276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118817,서울특별시 은평구 구산동,382,382.0,0.0,갈현현대,59.94,200707,12,11,1998,...,2013-06-04 16:18:51.0,Y,N,126.905638,37.612962,2013-03-07 09:46:27.0,20000,서울특별시 은평구 구산동 382,126.905543,37.612989
1118818,서울특별시 은평구 구산동,382,382.0,0.0,갈현현대,59.94,200708,25,10,1998,...,2013-06-04 16:18:51.0,Y,N,126.905638,37.612962,2013-03-07 09:46:27.0,20000,서울특별시 은평구 구산동 382,126.905543,37.612989
1118819,서울특별시 은평구 구산동,382,382.0,0.0,갈현현대,84.83,200708,31,20,1998,...,2013-06-04 16:18:51.0,Y,N,126.905638,37.612962,2013-03-07 09:46:27.0,28000,서울특별시 은평구 구산동 382,126.905543,37.612989
1118820,서울특별시 은평구 구산동,382,382.0,0.0,갈현현대,84.83,200709,15,8,1998,...,2013-06-04 16:18:51.0,Y,N,126.905638,37.612962,2013-03-07 09:46:27.0,29000,서울특별시 은평구 구산동 382,126.905543,37.612989


In [68]:
dt.drop(columns='지번주소', inplace=True)
dt

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,좌표X_2,좌표Y_2
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,124000,127.056859,37.476276
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,...,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,123500,127.056859,37.476276
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,201712,28,5,1987,...,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,91500,127.056859,37.476276
3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,3,4,1987,...,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,130000,127.056859,37.476276
4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,8,2,1987,...,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,117000,127.056859,37.476276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118817,서울특별시 은평구 구산동,382,382.0,0.0,갈현현대,59.94,200707,12,11,1998,...,의무,2013-06-04 16:18:51.0,Y,N,126.905638,37.612962,2013-03-07 09:46:27.0,20000,126.905543,37.612989
1118818,서울특별시 은평구 구산동,382,382.0,0.0,갈현현대,59.94,200708,25,10,1998,...,의무,2013-06-04 16:18:51.0,Y,N,126.905638,37.612962,2013-03-07 09:46:27.0,20000,126.905543,37.612989
1118819,서울특별시 은평구 구산동,382,382.0,0.0,갈현현대,84.83,200708,31,20,1998,...,의무,2013-06-04 16:18:51.0,Y,N,126.905638,37.612962,2013-03-07 09:46:27.0,28000,126.905543,37.612989
1118820,서울특별시 은평구 구산동,382,382.0,0.0,갈현현대,84.83,200709,15,8,1998,...,의무,2013-06-04 16:18:51.0,Y,N,126.905638,37.612962,2013-03-07 09:46:27.0,29000,126.905543,37.612989


In [69]:
dt.to_csv(target_filepath, encoding='utf-8-sig', index=False)